In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [2]:
# !pip install playwright

In [3]:
# !pip install tabulate

In [4]:
import asyncio
import json
import os
import nest_asyncio
import pprint
import base64
from io import BytesIO
import pandas as pd
from playwright.async_api import async_playwright
from openai import OpenAI
from PIL import Image
from tabulate import tabulate
from IPython.display import display, HTML, Markdown
from pydantic import BaseModel
from helper import visualizeCourses

In [5]:
nest_asyncio.apply()

In [6]:
class WebScraperAgent:
    def __init__(self):
        self.playwright = None
        self.browser = None
        self.page = None

    async def init_browser(self):
        try:
            self.playwright = await async_playwright().start()
            self.browser = await self.playwright.chromium.launch(
                headless=True,
                args=[
                    "--disable-dev-shm-usage",
                    "--no-sandbox",
                    "--disable-setuid-sandbox",
                    "--disable-accelerated-2d-canvas",
                    "--disable-gpu",
                    "--no-zygote",
                    "--disable-audio-output",
                    "--disable-software-rasterizer",
                    "--disable-webgl",
                    "--disable-web-security",
                    "--disable-features=LazyFrameLoading",
                    "--disable-features=IsolateOrigins",
                    "--disable-background-networking"
                ]
            )
            self.page = await self.browser.new_page()
        
        except Exception as e:
            print(f"❌ Browser initialization failed: {e}")
            await self.close()  # Clean up partial initialization
            raise  # Re-raise to halt execution


    async def scrape_content(self, url):
        if not self.page or self.page.is_closed():
            await self.init_browser()
        await self.page.goto(url, wait_until="load")
        await self.page.wait_for_timeout(2000)  # Wait for dynamic content
        return await self.page.content()

    async def take_screenshot(self, path="screenshot.png"):
        await self.page.screenshot(path=path, full_page=True)
        return path
    async def screenshot_buffer(self):
        screenshot_bytes = await self.page.screenshot(type="png", full_page=False)
        return screenshot_bytes

    async def close(self):
        try:
            if self.page and not self.page.is_closed():
                await self.page.close()
            if self.browser:
                await self.browser.close()
            if self.playwright:
                await self.playwright.stop()
        except Exception as e:
            print(f"⚠️ Cleanup error (ignored): {e}")
        finally:
            self.page = None
            self.browser = None
            self.playwright = None

In [7]:
scraper = WebScraperAgent()

In [8]:
class DeeplearningCourse(BaseModel):
    title: str
    description: str
    presenter: list[str]
    imageUrl: str
    courseURL: str

class DeeplearningCourseList(BaseModel):
    courses: list[DeeplearningCourse]

In [9]:
client = OpenAI()

In [10]:
async def process_with_llm(html, instructions, truncate = False):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini-2024-07-18",
        messages=[{
            "role": "system",
            "content": f"""
            You are an expert web scraping agent. Your task is to:
            Extract relevant information from this HTML to JSON 
            following these instructions:
            {instructions}
            
            Extract the title, description, presenter, 
            the image URL and course URL for each of 
            all the courses for the deeplearning.ai website

            Return ONLY valid JSON, no markdown or extra text."""
        }, {
            "role": "user",
            "content": html[:150000]  # Truncate to stay under token limits
        }],
        temperature=0.1,
        response_format=DeeplearningCourseList,
        )
    return completion.choices[0].message.parsed

In [11]:
async def webscraper(target_url, instructions):
    result = None
    screenshot = None

    try:
        print("Initializing browser...")
        await scraper.init_browser()  # Explicit initialization

        # Scrape content and capture screenshot
        print("Extracting HTML Content \n")
        html_content = await scraper.scrape_content(target_url)

        print("Taking Screenshot \n")
        screenshot = await scraper.screenshot_buffer()
        # Process content

        print("Processing..")
        result: DeeplearningCourseList = await process_with_llm(html_content, instructions, False)
        print("\nGenerated Structured Response")

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        
    finally:
        await scraper.close()

    return result, screenshot

In [12]:
target_url = "https://www.deeplearning.ai/courses"  # Deeplearning AI courses
base_url="https://deeplearning.ai"

In [13]:
instructions = """
    Get all the courses
"""
result, screenshot = await webscraper(target_url, instructions)

Initializing browser...
❌ Browser initialization failed: 
❌ Error: 


In [14]:
print(result)

None
